<a href="https://colab.research.google.com/github/Mech1907/OPPS_PYTHONS_EPAT/blob/main/copy_of_oop_mj_strategy_multi_stock_multi_ma_cross_over_04_01_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install pyfolio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
import pyfolio as pf


/usr/local/lib/python3.8/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [31]:
import yfinance as yf

# Get the historical data for Microsoft's stock
#msft = yf.Ticker("MSFT")
historical_data = yf.Ticker("MSFT").history(period="max")

# Print the first few rows of the data
print(historical_data.head())


                               Open      High       Low     Close      Volume  \
Date                                                                            
1986-03-13 00:00:00-05:00  0.055380  0.063524  0.055380  0.060809  1031788800   
1986-03-14 00:00:00-05:00  0.060809  0.064067  0.060809  0.062980   308160000   
1986-03-17 00:00:00-05:00  0.062980  0.064609  0.062980  0.064067   133171200   
1986-03-18 00:00:00-05:00  0.064067  0.064610  0.061894  0.062437    67766400   
1986-03-19 00:00:00-05:00  0.062437  0.062980  0.060809  0.061351    47894400   

                           Dividends  Stock Splits  
Date                                                
1986-03-13 00:00:00-05:00        0.0           0.0  
1986-03-14 00:00:00-05:00        0.0           0.0  
1986-03-17 00:00:00-05:00        0.0           0.0  
1986-03-18 00:00:00-05:00        0.0           0.0  
1986-03-19 00:00:00-05:00        0.0           0.0  


In [41]:
class backtesting_crossover:
  def __init__(self, Ticker,period, SMA_Short, SMA_Long):
    self.Ticker = Ticker
    self.period = period
    self.SMA_Short = SMA_Short
    self.SMA_Long = SMA_Long
    self.fetch_data()
    self.indicators()
    self.signal()
    self.position()
    self.returns()
  def fetch_data(self):
    self.data = yf.Ticker(self.Ticker).history(period="max")
  def indicators(self):
   self.data["SMA_Short"] = self.data["Close"].rolling(window = self.SMA_Short, center = False).mean()
   self.data["SMA_Long"] = self.data["Close"].rolling(window = self.SMA_Long, center= False).mean()
   self.data["SMA_Short_Pre_Day"] = self.data["SMA_Short"].shift()
   self.data["SMA_Long_Pre_Day"] =self.data["SMA_Long"].shift()
   self.data.dropna(inplace = True)
  def signal(self):
   self.data["signal"] = np.where((self.data["SMA_Short"] > self.data["SMA_Long"]) 
                         & (self.data["SMA_Short_Pre_Day"] < self.data["SMA_Long_Pre_Day"]),1,0)
   self.data["signal"] = np.where((self.data["SMA_Short"] < self.data["SMA_Long"]) 
                         & (self.data["SMA_Short_Pre_Day"] > self.data["SMA_Long_Pre_Day"]),-1, self.data["signal"]) 

  def position(self):
    self.data["position"] = self.data["signal"].replace(to_replace = 0, method = "ffill")

  def returns(self):
   self.data["bnh_returns"]  = np.log(self.data["Close"]/self.data["Close"].shift(1))
   self.data["Strategy_Return"] = self.data["bnh_returns"]* self.data["position"].shift(1)
   print("Total_return:=", self.data["Strategy_Return"].cumsum()[-1])
   return self.data["Strategy_Return"].cumsum()[-1]
  def analysis(self):
     self.data[["SMA_Short", "SMA_Long", "position"]].plot(figsize = (15,10), secondary_y = "position", grid = True)
     plt.title("Checking if the position are generated properly")
     plt.show()
     self.data[["bnh_returns", "Strategy_Return"]].cumsum().plot(figsize = (15,6), secondary_y = "position", grid =True)

In [ ]:
#end_date = dt.datetime(2020,6,30).date()
#start_date = end_date-pd.Timedelta(days = 3*252)

In [42]:
nifty_10_20 = backtesting_crossover("AAPL","max",10,20)

Total_return:= 5.5828063355731805


In [34]:
AAPL_10_20 = backtesting_crossover("AAPL", "max", 10,20)

KeyError: ignored

In [44]:
class backtesting_EMA_crossover(backtesting_crossover):
  def __init__(self, Ticker, period, EMA_Short, EMA_Long):
    self.Ticker = Ticker
    self.period = period
    self.EMA_Short = EMA_Short
    self.EMA_Long = EMA_Long
    self.fetch_data()
    self.indicators()
    self.signal()
    self.position()
    self.returns()
  def fetch_data(self):
    self.data = yf.Ticker(self.Ticker).history(period="max")
  def indicators(self):
   self.data["EMA_Short"] = self.data["Close"].ewm(span = self.EMA_Short, adjust = False).mean()
   self.data["EMA_Long"] = self.data["Close"].ewm(span= self.EMA_Long, adjust= False).mean()
   self.data["EMA_Short_Pre_Day"] = self.data["EMA_Short"].shift()
   self.data["EMA_Long_Pre_Day"] =self.data["EMA_Long"].shift()
   self.data.dropna(inplace = True)
  @staticmethod
  def date_of_backtest():
    print("Date of Backtest:=", dt.datetime.now().date())

  def signal(self):
   self.data["signal"] = np.where((self.data["EMA_Short"] > self.data["EMA_Long"]) 
                         & (self.data["EMA_Short_Pre_Day"] < self.data["EMA_Long_Pre_Day"]),1,0)
   self.data["signal"] = np.where((self.data["EMA_Short"] < self.data["EMA_Long"]) 
                         & (self.data["EMA_Short_Pre_Day"] > self.data["EMA_Long_Pre_Day"]),-1, self.data["signal"]) 

  def position(self):
    self.data["position"] = self.data["signal"].replace(to_replace = 0, method = "ffill")

  def returns(self):
   self.data["bnh_returns"]  = np.log(self.data["Adj Close"]/self.data["Adj Close"].shift(1))
   self.data["Strategy_Return"] = self.data["bnh_returns"]* self.data["position"].shift(1)
   print("Total_return:=", self.data["Strategy_Return"].cumsum()[-1])
   return self.data["Strategy_Return"].cumsum()[-1]
  def analysis(self):
     self.data[["EMA_Short", "EMA_Long", "position"]].plot(figsize = (15,10), secondary_y = "position", grid = True)
     plt.title("Checking if the position are generated properly")
     plt.show()
     self.data[["bnh_returns", "Strategy_Return"]].cumsum().plot(figsize = (15,6), secondary_y = "position", grid =True)

In [46]:
import csv

In [49]:
Short_MA_list = [5,9,13,26, 50,100]
Long_MA_list = [5,9, 13,26,50,100,200]
# Open the CSV file
with open('/content/NIFTY_50.csv', 'r') as f:
  # Create a CSV reader
  reader = csv.reader(f)
  # Read the rows of the file
  rows = [row for row in reader]
# Convert each row to a string
stock_list = [','.join(row) for row in rows]

stock_name = []
Short_MA = []
Long_MA = []
Net_Return = []
for i in Short_MA_list:
  for j in Long_MA_list:
    for k in stock_list:
      print("For", i, j, k)
      a = backtesting_crossover(k, "max", i, j)
      Short_MA.append(i)
      Long_MA.append(j)
      stock_name.append(k)
      Net_Return.append(a.returns())

For 5 5 ADANIENT.NS


/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total_return:= 0.0
Total_return:= 0.0
For 5 5 ADANIPORTS.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 APOLLOHOSP.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 ASIANPAINT.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 AXISBANK.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 BAJAJ-AUTO.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 BAJFINANCE.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 BAJAJFINSV.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 BPCL.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 BHARTIARTL.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 BRITANNIA.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 CIPLA.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 COALINDIA.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 DIVISLAB.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 DRREDDY.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 EICHERMOT.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 GRASIM.NS
Total_return:= 0.0
Total_return:= 0.0
For 5 5 HCLTECH.NS
Total_re

/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total_return:= 8.920265082706242
Total_return:= 8.920265082706242
For 5 9 ADANIPORTS.NS
Total_return:= -0.7543377346293887
Total_return:= -0.7543377346293887
For 5 9 APOLLOHOSP.NS
Total_return:= -3.6175512356767476
Total_return:= -3.6175512356767476
For 5 9 ASIANPAINT.NS
Total_return:= 0.27629476908342926
Total_return:= 0.27629476908342926
For 5 9 AXISBANK.NS
Total_return:= -0.5714944278666335
Total_return:= -0.5714944278666335
For 5 9 BAJAJ-AUTO.NS
Total_return:= -0.584731050786994
Total_return:= -0.584731050786994
For 5 9 BAJFINANCE.NS
Total_return:= -1.0696961101347313
Total_return:= -1.0696961101347313
For 5 9 BAJAJFINSV.NS
Total_return:= 6.817496644097582
Total_return:= 6.817496644097582
For 5 9 BPCL.NS
Total_return:= 0.07782465973521342
Total_return:= 0.07782465973521342
For 5 9 BHARTIARTL.NS
Total_return:= -2.2098482171421137
Total_return:= -2.2098482171421137
For 5 9 BRITANNIA.NS
Total_return:= 1.347106815427486
Total_return:= 1.347106815427486
For 5 9 CIPLA.NS
Total_return:= -

...

Total_return:= -1.518757420821202
Total_return:= -1.518757420821202
For 50 26 UPL.NS
Total_return:= 0.8549830740290502
Total_return:= 0.8549830740290502
For 50 26 ULTRACEMCO.NS
Total_return:= -1.185714317552111
Total_return:= -1.185714317552111
For 50 26 WIPRO.NS


/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total_return:= -4.9000586061636575
Total_return:= -4.9000586061636575
For 50 50 ADANIENT.NS


/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total_return:= 0.0
Total_return:= 0.0
For 50 50 ADANIPORTS.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 APOLLOHOSP.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 ASIANPAINT.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 AXISBANK.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 BAJAJ-AUTO.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 BAJFINANCE.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 BAJAJFINSV.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 BPCL.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 BHARTIARTL.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 BRITANNIA.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 CIPLA.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 COALINDIA.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 DIVISLAB.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 DRREDDY.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 EICHERMOT.NS
Total_return:= 0.0
Total_return:= 0.0
For 50 50 GRASIM.NS
Total_return:= 0.0
Total_return:=

In [50]:
results = pd.DataFrame({"Long_MA" : Long_MA, "Short_MA": Short_MA, "stock_name": stock_name, "Net_Return": Net_Return})
results

,Long_MA,Short_MA,stock_name,Net_Return
0,5,5,ADANIENT.NS,0.000000
1,5,5,ADANIPORTS.NS,0.000000
2,5,5,APOLLOHOSP.NS,0.000000
3,5,5,ASIANPAINT.NS,0.000000
4,5,5,AXISBANK.NS,0.000000
...,...,...,...,...
2095,200,100,TECHM.NS,0.196058
2096,200,100,TITAN.NS,0.326047
2097,200,100,UPL.NS,0.796669
2098,200,100,ULTRACEMCO.NS,1.131890


In [51]:
results.sort_values(by = "Net_Return", ascending= False)

,Long_MA,Short_MA,stock_name,Net_Return
1444,5,50,TATASTEEL.NS,54.139792
1494,9,50,TATASTEEL.NS,52.874058
1794,5,100,TATASTEEL.NS,49.672683
1844,9,100,TATASTEEL.NS,48.973360
1094,5,26,TATASTEEL.NS,43.948027
...,...,...,...,...
194,26,5,TATASTEEL.NS,-43.948028
644,100,9,TATASTEEL.NS,-48.973357
294,100,5,TATASTEEL.NS,-49.672685
594,50,9,TATASTEEL.NS,-52.874058


In [52]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")


In [53]:
results.to_csv(f"output_{timestamp}.csv", index = False)

In [54]:
!pip install -U -q PyDrive

In [55]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
